In [2]:
import threading 
import time
import numpy as np
import cv2

#https://stackoverflow.com/questions/63345956/python-opencv-mutithreading-with-opecv-video-streaming

# --- functions ---

running = True

def print_hello():
    while running:
        print("Hello World")
        time.sleep(3)

# --- main ---

t1 = threading.Thread(target=print_hello)  
t1.start()

# --- loop ---

cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- after loop ---

running = False # to stop loop in thread
t1.join()

cap.release()
cv2.destroyAllWindows()

Hello World
Hello World
Hello World
Hello World


In [1]:
from threading import Thread
import cv2, time
 
class VideoStreamWidget(object):
    def __init__(self, src=0):
        self.capture = cv2.VideoCapture(src)
        # Start the thread to read frames from the video stream
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True
        self.thread.start()

    def update(self):
        # Read the next frame from the stream in a different thread
        while True:
            if self.capture.isOpened():
                (self.status, self.frame) = self.capture.read()
            time.sleep(.01)
    
    def show_frame(self):
        # Display frames in main program
        cv2.imshow('frame', self.frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            self.capture.release()
            cv2.destroyAllWindows()
            exit(1)

if __name__ == '__main__':
    video_stream_widget = VideoStreamWidget()
    while True:
        try:
            video_stream_widget.show_frame()
        except AttributeError:
            pass

KeyboardInterrupt: 

## Simple threading

https://www.youtube.com/watch?v=IEEhzQoKtQU

In [10]:
import threading
import time

start = time.perf_counter()


def do_something():
    print('Sleeping 1 second')
    time.sleep(1)
    print('Done Sleeping.....')

threads = []

for _ in range(10):
    t = threading.Thread(target=do_something)
    t.start()
    threads.append(t)

for thread in threads:
    thread.join()   

finish = time.perf_counter()

print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1 second
Sleeping 1 second
Sleeping 1 second
Sleeping 1 second
Sleeping 1 second
Sleeping 1 second
Sleeping 1 second
Sleeping 1 second
Sleeping 1 second
Sleeping 1 second
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Finished in 1.02 second(s)


## Passing seconds to sleep as an argumet

In [12]:
import threading
import time

start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    print('Done Sleeping.....')

threads = []

for _ in range(10):
    t = threading.Thread(target=do_something, args=[1.5])
    t.start()
    threads.append(t)

for thread in threads:
    thread.join()   

finish = time.perf_counter()

print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Done Sleeping.....Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....
Done Sleeping.....

Done Sleeping.....
Finished in 1.53 second(s)


## Concurrent method runing f1 process manually

In [17]:
import concurrent.futures
import time

start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return 'Done Sleeping...'


with concurrent.futures.ThreadPoolExecutor() as executor:
    f1 = executor.submit(do_something, 1)
    f2 = executor.submit(do_something, 1)
    print(f1.result())
    print(f2.result())


finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 1 second(s)...
Sleeping 1 second(s)...
Done Sleeping...
Done Sleeping...
Finished in 1.01 second(s)


In [19]:
import concurrent.futures
import time

start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'


with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5,4,3,2,1]
    results = [executor.submit(do_something, sec) for sec in secs]

    for f in concurrent.futures.as_completed(results):
        print(f.result())
    


finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 5 second(s)...
Sleeping 4 second(s)...
Sleeping 3 second(s)...
Sleeping 2 second(s)...
Sleeping 1 second(s)...
Done Sleeping...1
Done Sleeping...2
Done Sleeping...3
Done Sleeping...4
Done Sleeping...5
Finished in 5.01 second(s)


In [25]:
import concurrent.futures
import time

start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'


with concurrent.futures.ProcessPoolExecutor() as executor:
    secs = [5, 4, 3, 2, 1]
    results = [executor.submit(do_something, sec) for sec in secs]
    #results = executor.map(do_something, secs)

    #for f in concurrent.futures.as_completed(results):
    #    print(f.result())


finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

## Multi-Threading use data from a thread in another thread

In [31]:
import time
import threading

available = False

def thread1():
    global available
    while True:
        # TODO: call API
        # --------------

        available = True # set available True after API call
        time.sleep(5) # perform API calls after every 5 seconds
        print(available)

def thread2():
    while True:
        # TODO: perform ping
        # --------------

        # perform ping request after every 5 seconds
        time.sleep(5)

if __name__ == "__main__":
    t1 = threading.Thread(target=thread1, name="thread1")
    t2 = threading.Thread(target=thread2, name="thread2")

    t1.start()
    t1.join()

    while not available:
        time.sleep(0.1)
    else:
        t2.start()

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


## Multithreading sharing variable
https://stackoverflow.com/questions/17774768/python-creating-a-shared-variable-between-threads

In [2]:
import numpy as np
np.random.randint(10,size=np.random.randint(10))


array([5, 7, 9, 1, 2, 1])

In [1]:
import threading
import time
import numpy as np
import datetime


c = threading.Condition()
flag = 0      #shared between Thread_A and Thread_B
val = 0
count = 0

class Thread_A(threading.Thread):
    def __init__(self, name):
        threading.Thread.__init__(self)
        self.name = name

    def run(self):
        global flag
        global val     #made global here
        global count     
        while count < 5:
            c.acquire()
            if flag == 0:
                print("A: val=" + str(val))
                #time.sleep(0.1)
                flag = 1
                val = np.random.randint(10,size=np.random.randint(10))
                c.notify_all()
            else:
                c.wait()
            c.release()


class Thread_B(threading.Thread):
    def __init__(self, name):
        threading.Thread.__init__(self)
        self.name = name

    def run(self):
        global flag
        global val    #made global here
        global count 
        while count < 5:
            c.acquire()
            if flag == 1:
                timestamp = datetime.datetime.now().strftime('%y-%m-%d %H:%M:%S.%f')
                print("Time: {0}  B: val= {1}".format(timestamp, str(val)))
                time.sleep(2)
                flag = 0
                val = 0
                count += 1
                c.notify_all()
            else:
                c.wait()
            c.release()



a = Thread_A("myThread_name_A")
b = Thread_B("myThread_name_B")


a.start()
threading.Timer(1, b)#.start()
b.start()


a.join()
b.join()

A: val=0
Time: 22-08-09 07:18:45.545099  B: val= [0 6 4 7 9 3 8 5]
A: val=0
Time: 22-08-09 07:18:47.554558  B: val= [6]
A: val=0
Time: 22-08-09 07:18:49.562732  B: val= [3 9]
A: val=0
Time: 22-08-09 07:18:51.565677  B: val= []
A: val=0
Time: 22-08-09 07:18:53.576557  B: val= [5 0 7 9 5 4]


In [2]:
import datetime

In [6]:
timestamp = datetime.datetime.now().strftime('%y-%m-%d %H:%M:%S.%f')

print('Time: {0}'.format(timestamp))

Time: 22-08-08 07:37:06.420565
